In [1]:
from pycparser import c_parser, c_ast

In [2]:
with open('../kernels_lore/k4/k4.txt', 'r') as fin:
    code = fin.read()
    parser = c_parser.CParser()
    ast = parser.parse(code)

In [5]:
ast.ext = [3] + ast.ext

In [7]:
ast.ext[3].coord.

In [152]:
def max_set(set):
    return set.pop if len(set) == 1 else 'max(' + ','.join(set) + ')'


def q(n, maxs={}):
    if type(n) is str:
        return max_set(maxs[n]) if n in maxs else n
    if type(n) is c_ast.ID:
        return list(maxs[n.name])[0] if n.name in maxs else n.name
    elif type(n) is c_ast.BinaryOp:
        return q(n.left, maxs) + n.op + q(n.right, maxs)
    elif type(n) is c_ast.Constant:
        return n.value
    else:
        return '42'
    
    

class ArrayRefVisitor(c_ast.NodeVisitor):
    def __init__(self, maxs):
        self.refs = {}
        self.maxs = maxs
        
    def visit_ArrayRef(self, node):
        n = node.name.name
        s = node.subscript
        
        if n in self.refs:
            self.refs[n].add(q(s, self.maxs))
        else:
            self.refs[n] = set([q(s, self.maxs)])
            
            
class AssignmentVisitor(c_ast.NodeVisitor):
    def __init__(self, maxs):
        self.refs = {}
        self.maxs = maxs
        
    def visit_Assignment(self, node):
        if node.op == '=':
            l = node.lvalue
            r = node.rvalue

            if type(l) is c_ast.ID:
                if l.name in self.maxs:
                    self.maxs[l.name].add(q(r, self.maxs))
                else:
                    self.maxs[l.name] = set([q(r, self.maxs)])
            

class ForVisitor(c_ast.NodeVisitor):
    def __init__(self):
        self.maxs = {}
            
    def visit_For(self, node):
        c = node.cond
        if type(c) is not c_ast.BinaryOp:
            return
        
        v = c.left
        m = c.right
        
        if type(v) is not c_ast.ID:
            return
        
        if v.name in self.maxs:
            self.maxs[v.name].add(q(m))
        else:
            self.maxs[v.name] = set([q(m)])
            
        c_ast.NodeVisitor.generic_visit(self, node)
        
        
cv = ForVisitor()
cv.visit(ast)    
print(cv.maxs)
        
cv = AssignmentVisitor(cv.maxs)
cv.visit(ast)
print(cv.maxs)
        
cv = ArrayRefVisitor(cv.maxs)
cv.visit(ast)
print(cv.refs)
        

{'i': {'n'}, 'j': {'n'}}
{'i': {'n', '0'}, 'j': {'n', '0'}, 'q': {'n'}}
{'A': {'n*n+n'}, 'D': {'n'}}


In [147]:
q('q+1', cv.maxs)

'q+1'

In [125]:
ast.show()

FileAST: 
  Typedef: long_long, [], ['typedef']
    TypeDecl: long_long, []
      IdentifierType: ['double']
  Decl: i, [], [], []
    TypeDecl: i, []
      IdentifierType: ['int']
  Decl: n, [], [], []
    TypeDecl: n, []
      IdentifierType: ['int']
  Decl: j, [], [], []
    TypeDecl: j, []
      IdentifierType: ['int']
  Decl: q, [], [], []
    TypeDecl: q, []
      IdentifierType: ['int']
  Decl: A, [], [], []
    PtrDecl: []
      TypeDecl: A, []
        IdentifierType: ['double']
  Decl: D, [], [], []
    PtrDecl: []
      TypeDecl: D, []
        IdentifierType: ['double']
  FuncDef: 
    Decl: loop, [], [], []
      FuncDecl: 
        ParamList: 
          Decl: set, [], [], []
            TypeDecl: set, []
              IdentifierType: ['int']
          Decl: values, [], [], []
            PtrDecl: []
              TypeDecl: values, []
                IdentifierType: ['long_long']
        TypeDecl: loop, []
          IdentifierType: ['int']
    Compound: 
      Pragma: scop
  

In [150]:
len(set([3]))

1